In [12]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
from pathlib import Path
sys.path.insert(0,"..")
from src.evaluation.evaluate import Sent2Sent, match,match_ignore_case,match_ignore_case_punc,match_ignore_case_punc_keep_hyphen

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# home = Path(os.getcwd()).parent.joinpath('predictions_out')
# predictions_list = []
# for dirpath, dirnames, filenames in os.walk(home):
#     for name in filenames:
#         if name.endswith('.out') or name.endswith('.tsv'):
#             predictions_list.append(f'{dirpath}/{name}')
# sorted(predictions_list,reverse=True)

In [4]:
test = pd.read_csv(f'{home.parent}/data/processed/test.tsv',delimiter='\t')

In [11]:
dev = pd.read_csv(f'{home.parent}/data/processed/dev.tsv',delimiter='\t')

In [8]:
train = pd.read_csv(f'{home.parent}/data/processed/train.tsv',delimiter='\t')

In [16]:
allsets = pd.concat([train,dev,test])
allsets.drop_duplicates('sentID').value_counts('splits',normalize=True)

splits
train    0.800297
dev      0.100022
test     0.099680
dtype: float64

In [6]:
evalscorestest = []
exceptiontest = []
for p in predictions:
    with open(p,'r') as i:
        prediction = [x.strip() for x in i.readlines()]
        evaldf = evaluate(test,prediction,eval_funcs=[match,match_ignore_case,match_ignore_case_punc]) #,match_ignore_case,match_ignore_case_punc # match_ignore_case_punc_keep_hyphen
        name = p.split("/")[-1]
        print(name)
        evaldf.to_csv(f'{home.parent}/evaluation/test/eval-{name}',sep='\t',index=False)
        for m in [col for col in evaldf if col.startswith('match')]:
            score = evaldf[m].value_counts(normalize=True)
            try:
                print(score[True].to_string())
            except:
                exceptiontest.append((name,m,score))
            evalscorestest.append((name,m,score))

line2line-test_aligned_rulesmorph.out
line2line-test_aligned_mlesimple.out


In [7]:
names = {}
for name,match,score in evalscorestest:
    if name in names:
        names[name].update({match:score[True]})
    else:
        names[name] = {match:score[True]}
    
pd.DataFrame(names).T

,match,match_ignore_case,match_ignore_case_punc
line2line-test_aligned_rulesmorph.out,0.886986,0.914551,0.928771
line2line-test_aligned_mlesimple.out,0.890400,0.916396,0.930611


In [5]:
# evalscores = []
# exception = []
# for p in predictions:
#     with open(p,'r') as i:
#         prediction = [x.strip() for x in i.readlines()]
#         evaldf = evaluate(test,prediction,eval_funcs=[match,match_ignore_case,match_ignore_case_punc]) #,match_ignore_case,match_ignore_case_punc # match_ignore_case_punc_keep_hyphen
#         name = p.split("/")[-1]
#         print(name)
#         evaldf.to_csv(f'{home.parent}/evaluation/dev/eval-{name}',sep='\t',index=False)
#         for m in [col for col in evaldf if col.startswith('match')]:
#             score = evaldf[m].value_counts(normalize=True)
#             try:
#                 print(score[True].to_string())
#             except:
#                 exception.append((name,m,score))
#             evalscores.append((name,m,score))

In [11]:
names = {}
for name,match,score in evalscores:
    if name in names:
        names[name].update({match:score[True]})
    else:
        names[name] = {match:score[True]}
    
pd.DataFrame(names).T

,match,match_ignore_case,match_ignore_case_punc
line2line-dev_aligned_simple.out,0.889049,0.915165,0.929371
line2line-dev_aligned_morph.out,0.889373,0.915410,0.929611


In [217]:
evalscores2 = []
for e in os.listdir(home.parent.joinpath('evaluation/dev')):
    name = e
    print(e)
    evaldf = pd.read_csv(home.parent.joinpath(f'evaluation/dev/{e}'),delimiter='\t')
    for m in [col for col in evaldf if col.startswith('match')]:
        score = evaldf[m].value_counts(normalize=True)
        print(score[True].to_string())
        evalscores2.append((name,m,score))

In [216]:
names2 = {}
for name,match,score in evalscores2:
    if name in names2:
        names2[name].update({match:score[True]})
    else:
        names2[name] = {match:score[True]}
    
pd.DataFrame(names2).T.sort_index().to_clipboard()#.reset_index(drop=True)==pd.DataFrame(names).T.sort_values('match').reset_index(drop=True)

In [7]:
home = Path(os.getcwd()).parent
with open(home.joinpath('predictions_out/translit_morph/dev/rules-morph.tsv'),'r') as f:
    rules_morph = [line.strip() for line in f.readlines()]

dev = pd.read_csv(home.joinpath('data/processed/dev.tsv'),delimiter='\t')

In [8]:
e = Sent2Sent(dev,rules_morph,'rules-morph')
e.calculate_word2word()


Misaligned gold/prediction sentID: 20388


KeyboardInterrupt: 

In [22]:
dev

,sentID,rom,ar,rom_raw,ar_raw,comb.tag,recID,subtag,tag,link,source,splits
0,0,"Rāfiʻī, ʻAbd al-Raḥmān.",رافعي، عبد الرحمن.,"Rāfiʻī, ʻAbd al-Raḥmān.",رافعي، عبد الرحمن.,100a,umich-1,a,100,0,umich,dev
1,1,Muqaddimāt thawrat 23 Yūlyū sanat 1952 :,مقدمات ثورة 23 يوليوا سنة 1952 :,Muqaddimāt thawrat 23 Yūlyū sanat 1952 :,مقدمات ثورة 23 يوليوا سنة 1952 :,245a,umich-1,a,245,4,umich,dev
2,2,"al-kifāḥ fī al-qitāl, ḥarīq al-Qāhirah, wazārā...",الكفاح في القتال، حريق القاهرة، وزارات الموظفي...,"al-kifāḥ fī al-qitāl, ḥarīq al-Qāhirah, wazārā...",الكفاح في القتال، حريق القاهرة، وزارات الموظفي...,245b,umich-1,b,245,4,umich,dev
3,3,bi-qalam ʻAbd al-Raḥmān al-Rifāʻī.,بقلم عبد الرحمن الرفاعي.,bi-qalam ʻAbd al-Raḥmān al-Rifāʻī.,بقلم عبد الرحمن الرفاعي.,245c,umich-1,c,245,4,umich,dev
4,4,Muqaddimāt thawrat thalāthah wa-ʻishrīn Yūlyū ...,مقدمات ثورة ثلاثة وعشرين يوليوا سنة 1952,Muqaddimāt thawrat thalāthah wa-ʻishrīn Yūlyū ...,مقدمات ثورة ثلاثة وعشرين يوليوا سنة 1952,246a,umich-1,a,246,4,umich,dev
...,...,...,...,...,...,...,...,...,...,...,...,...
59968,599510,"Muʼassasat al-Yarmūk lil-Thaqāfah wa-al-Iʻlām,",مؤسسة اليرموك للثقافة والاعلام،,"Muʼassasat al-Yarmūk lil-Thaqāfah wa-al-Iʻlām,",مؤسسة اليرموك للثقافة والاعلام،,260b,loc-90065,b,260,6,loc,dev
59969,599556,al-Qānūn al-asāsī lil-ṣihāfah /,القانون الاساسي للصحافة /,al-Qānūn al-asāsī lil-ṣihāfah /,القانون الاساسي للصحافة /,245a,loc-90075,a,245,4,loc,dev
59970,599557,taʻrīb ʻAbd al-Qādir al-Mīlādī.,تعريب عبد القادر الميلادي.,taʻrīb ʻAbd al-Qādir al-Mīlādī.,تعريب عبد القادر الميلادي.,245c,loc-90075,c,245,4,loc,dev
59971,599558,"al-Jumhūrīyah al-Islāmīyah al-Mūrītānīyah, al-...",الجمهورية الاسلامية الموريتانية، المدرسة الوطن...,al-Jumhūrīyah al-Islāmīyah al-Mūrītānī...,الجمهورية الاسلامية الموريتانية، المدرسة الوطن...,260b,loc-90075,b,260,6,loc,dev


In [13]:
ext = pd.read_csv(home.joinpath('data/extracted_lines/extracted_lines.tsv'),delimiter='\t')

In [16]:
ext['comb.tag'] = ext.apply(lambda x: f"{x['tag']}{x['subtag']}",axis=1)


In [20]:
ext['comb.tag'].value_counts()[ext['comb.tag'].value_counts()>20]

245a    107666
245c    101408
100a     94566
260b     87652
260a     86905
         ...  
240p        24
700l        23
810b        22
1100        22
710c        21
Name: comb.tag, Length: 159, dtype: int64

In [ ]:
e.get_sent([20388,20389])

SyntaxError: can't use starred expression here (<ipython-input-35-a3f6cc391049>, line 7)

In [220]:
os.listdir(home.parent.joinpath('evaluation/dev'))

['eval-mle-translit_simple-size0.5.tsv',
 'eval-mle-translit_morph-size1.0.tsv',
 'eval-rules-morph.tsv',
 'eval-mle-translit_simple-size0.015625.tsv',
 'eval-line2line-dev.out',
 'eval-mle-translit_simple-size1.0.tsv',
 'eval-mle-translit_simple-size0.125.tsv',
 'eval-mle-translit_simple-size0.0625.tsv',
 'eval-mle-translit_simple-size0.25.tsv',
 'eval-rules-simple.tsv',
 'eval-mle-translit_simple-size0.03125.tsv']

In [11]:
dev[dev['sentID']==58543].values

array([], shape=(0, 12), dtype=object)

In [10]:
e = 'eval-rules-morph.tsv'
edf = pd.read_csv(home.parent.joinpath(f'evaluation/dev/{e}'),delimiter='\t')

In [35]:
# c1 = edf['prediction_tok'].apply(lambda x: 'ayy' in x)
c1 = edf['source_tok'].apply(lambda x: 'حشو' in str(x))
# c2 = edf['match_ignore_case_punc']==False 
edf[c1]#[120:180] #.value_counts('match_ignore_case')

,sentID,comb.tag,gold_tok,source_tok,prediction_tok,aligned_source/gold,aligned_source/prediction,aligned_gold/prediction,match,match_ignore_case,match_ignore_case_punc,match_ignore_case_punc_keep_hyphen
7833,20753,100a,"Ḥashwah,",حشوة،,"Ḥashwat,",True,True,True,False,False,False,False
7844,20756,245c,Ḥashwah,حشوه,ḥashshūhu,True,True,True,False,False,False,False
35190,79683,100a,"Ḥashwah,",حشوة،,"Ḥashwat,",True,True,True,False,False,False,False


In [25]:
test[test['ar'].apply(lambda x: 'معنى' in str(x))].values

array([[19849, 'Tashkīl al-maʻná al-shiʻrī :', 'تشكيل المعنى الشعري :',
        'Tashkīl al-maʻná al-shiʻrī :', 'تشكيل المعنى الشعري :', '245a',
        'umich-4013', 'a', 245, 4, 'umich', 'test'],
       [115743, 'Maʻná, ʻAbd al-Razzāq al-Sanūsī,',
        'معنى، عبد الرزاق السنوسي.',
        'Maʻná, ʻAbd al-Razzāq al-Sanūsī,',
        'معنى، عبد الرزاق السنوسي.', '100a', 'loc-3564', 'a', 100, 0,
        'loc', 'test'],
       [115746, 'ʻAbd al-Razzāq al-Sanūsī Maʻná.',
        'عبد الرزاق السنوسي معنى.',
        'ʻAbd al-Razzāq al-Sanūsī Maʻná.',
        'عبد الرزاق السنوسي معنى.', '245c', 'loc-3564', 'c', 245, 4,
        'loc', 'test'],
       [180684,
        'al-Maʻná wa-al-iʻrāb ʻinda al-naḥwīyīn wa-naẓarīyat al-ʻāmil /',
        'المعنى والاعراب عند النحويين ونظرية العامل /',
        'al-Maʻná wa-al-iʻrāb ʻinda al-naḥwīyīn wa-naẓarīyat al-ʻāmil /',
        'المعنى والاعراب عند النحويين ونظرية العامل /', '245a',
        'loc-14958', 'a', 245, 4, 'loc', 'test'],
  